## Setup Feast
Create a sample `rbac` project with local storage.

In [1]:
!rm -rf rbac
!feast init rbac


Creating a new Feast repository in /Users/dmartino/projects/AI/feast/feast/examples/rbac-local/rbac.



Update the `feature_store.yaml` with an `auth` section derived from the Keycloak setup file [.env](.env)

In [2]:
!cat .env

OIDC_SERVER_URL='http://0.0.0.0:9999'
REALM='rbac_example'
CLIENT_ID='app'
CLIENT_SECRET='REDACTED'
PASSWORD='password'


### Update the server YAML
Update the server YAML to use OIDC authorization

In [3]:
from dotenv import load_dotenv
import os
import yaml

def load_config_file(path):
    load_dotenv()

    with open(path, 'r') as file:
        config = yaml.safe_load(file) or {}
    return config

In [12]:
def update_config_with_auth(config, is_client=False):
    config['auth']={}
    config['auth']['type']='oidc'
    config['auth']['auth_discovery_url']=f"{os.getenv('OIDC_SERVER_URL')}/realms/{os.getenv('REALM')}/.well-known/openid-configuration"
    config['auth']['client_id']=os.getenv('CLIENT_ID')
    if is_client:
        config['auth']['client_secret']=os.getenv('CLIENT_SECRET')
        config['auth']['username']=''
        config['auth']['password']='password'

In [13]:
def update_config_file(path):
    with open(path, 'w') as file:
        yaml.safe_dump(config, file, default_flow_style=False)

In [14]:
config = load_config_file('rbac/feature_repo/feature_store.yaml')
update_config_with_auth(config)
update_config_file('rbac/feature_repo/feature_store.yaml')

In [15]:
!cat rbac/feature_repo/feature_store.yaml

auth:
  auth_discovery_url: http://0.0.0.0:9999/realms/rbac_example/.well-known/openid-configuration
  client_id: app
  type: oidc
entity_key_serialization_version: 2
online_store:
  path: data/online_store.db
  type: sqlite
project: rbac
provider: local
registry: data/registry.db


### Update the client YAML
Update the client YAML to use OIDC authorization

In [16]:
config = load_config_file('client/feature_store.yaml')
update_config_with_auth(config, is_client=True)
update_config_file('client/feature_store.yaml')

In [17]:
!cat client/feature_store.yaml

auth:
  auth_discovery_url: http://0.0.0.0:9999/realms/rbac_example/.well-known/openid-configuration
  client_id: app
  client_secret: REDACTED
  password: password
  type: oidc
  username: ''
entity_key_serialization_version: 2
offline_store:
  host: localhost
  port: 8815
  type: remote
online_store:
  path: http://localhost:6566
  type: remote
project: rbac
registry:
  path: localhost:6570
  registry_type: remote


### Apply the configuration

In [18]:
!feast -c rbac/feature_repo apply

Applying changes for project rbac
/Users/dmartino/projects/AI/feast/feast/sdk/python/feast/feature_store.py:562: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
Created project rbac
Created entity driver
Created feature view driver_hourly_stats
Created feature view driver_hourly_stats_fresh
Created on demand feature view transformed_conv_rate_fresh
Created on demand feature view transformed_conv_rate
Created feature service driver_activity_v1
Created feature service driver_activity_v3
Created feature service driver_activity_v2

Created sqlite table rbac_driver_hourly_stats_fresh
Created sqlite table rbac_driver_hourly_stats



### Validate permissions

There are no permissions after applying the example:

In [19]:
!feast -c rbac/feature_repo permissions list

NAME    TYPES    NAME_PATTERN    ACTIONS    ROLES    REQUIRED_TAGS


The `permissions check` command identifies the resources that have no permissions matching their type, name or tags.

In [20]:
!feast -c rbac/feature_repo permissions check

The following resources are not secured by any permission configuration:
NAME                         TYPE
driver                       Entity
driver_hourly_stats          FeatureView
driver_hourly_stats_fresh    FeatureView
transformed_conv_rate_fresh  OnDemandFeatureView
transformed_conv_rate        OnDemandFeatureView
driver_activity_v1           FeatureService
driver_activity_v3           FeatureService
driver_activity_v2           FeatureService
vals_to_add                  RequestSource
driver_stats_push_source     PushSource
driver_hourly_stats_source   FileSource
The following actions are not secured by any permission configuration (Note: this might not be a security concern, depending on the used APIs):
NAME                         TYPE                 UNSECURED ACTIONS
driver                       Entity               CREATE
                                                  DESCRIBE
                                                  UPDATE
                                     

### Applying permissions
Let's create some Permissions to cover basic scenarios.

First a simple permission to read the status of all the objects.

In [22]:
from feast import FeatureStore
from feast.feast_object import ALL_RESOURCE_TYPES
from feast.permissions.action import CRUD, AuthzedAction, ALL_ACTIONS
from feast.permissions.permission import Permission
from feast.permissions.policy import RoleBasedPolicy

In [23]:
store = FeatureStore("rbac/feature_repo")

In [24]:
read_permission = Permission(
    name="read_permission",
    types=ALL_RESOURCE_TYPES,
    policy=RoleBasedPolicy(roles=["reader"]),
    actions=AuthzedAction.DESCRIBE
)
store.registry.apply_permission(read_permission, store.project)

Now a specific permission to write online data (e.g. `materialize`) the `FeatureView`s whose name ends by `fresh`

In [25]:
from feast.feature_view import FeatureView
write_fresh_permission = Permission(
    name="write_fresh_permission",
    types=FeatureView,
    name_patterns=".*_fresh",
    policy=RoleBasedPolicy(roles=["fresh_writer"]),
    actions=AuthzedAction.WRITE_ONLINE
)
store.registry.apply_permission(write_fresh_permission, store.project)

Another one to match allow access to OFFLINE functions.

In [26]:
from feast.feature_view import FeatureView
from feast.feature_service import FeatureService
from feast.on_demand_feature_view import OnDemandFeatureView
offline_permission = Permission(
    name="offline_permission",
    types=[FeatureView, OnDemandFeatureView, FeatureService],
    policy=RoleBasedPolicy(roles=["batch_admin"]),
    actions= CRUD + [AuthzedAction.WRITE_OFFLINE, AuthzedAction.READ_OFFLINE]
)
store.registry.apply_permission(offline_permission, store.project)

Finally, ad `admin` permission to manage all the resources

In [27]:
admin_permission = Permission(
    name="admin_permission",
    types=ALL_RESOURCE_TYPES,
    policy=RoleBasedPolicy(roles=["store_admin"]),
    actions=ALL_ACTIONS
)
store.registry.apply_permission(admin_permission, store.project)

## Validate registered permissions

List all the permissions.

In [28]:
!feast -c rbac/feature_repo permissions list

NAME                    TYPES                NAME_PATTERN    ACTIONS        ROLES         REQUIRED_TAGS
read_permission         Project                              DESCRIBE       reader        -
                        FeatureView
                        OnDemandFeatureView
                        BatchFeatureView
                        StreamFeatureView
                        Entity
                        FeatureService
                        DataSource
                        ValidationReference
                        SavedDataset
                        Permission
write_fresh_permission  FeatureView          .*_fresh        WRITE_ONLINE   fresh_writer  -
offline_permission      FeatureView                          CREATE         batch_admin   -
                        OnDemandFeatureView                  DESCRIBE
                        FeatureService                       UPDATE
                                                             DELETE
                              

List all the resources matching each configured permission.

In [29]:
!feast -c rbac/feature_repo permissions list -v


The structure of the feast-permissions list --verbose command will be as in the following example:

For example: 

permissions
├── permission_1 ['role names list']
│   ├── FeatureView: ['feature view names']
│   ├── FeatureService: none
│   └── ..
├── permission_2 ['role names list']
└── ..

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
            
Permissions:

permissions
├── read_permission ['reader']
│   ├── FeatureView ['driver_hourly_stats_fresh', 'transformed_conv_rate_fresh', 'transformed_conv_rate', 'driver_hourly_stats']
│   ├── OnDemandFeatureView ['transformed_conv_rate_fresh', 'transformed_conv_rate']
│   ├── BatchFeatureView ['driver_hourly_stats_fresh', 'driver_hourly_stats']
│   ├── StreamFeatureView: none
│   ├── Entity: ['driver']
│   ├── FeatureService: ['driver_activity_v3', 'driver_activity_v2', 'driver_ac

Describe one of the permissions.

In [30]:
!feast -c rbac/feature_repo permissions describe admin_permission

spec:
  name: admin_permission
  types:
  - PROJECT
  - FEATURE_VIEW
  - ON_DEMAND_FEATURE_VIEW
  - BATCH_FEATURE_VIEW
  - STREAM_FEATURE_VIEW
  - ENTITY
  - FEATURE_SERVICE
  - DATA_SOURCE
  - VALIDATION_REFERENCE
  - SAVED_DATASET
  - PERMISSION
  actions:
  - CREATE
  - DESCRIBE
  - UPDATE
  - DELETE
  - READ_ONLINE
  - READ_OFFLINE
  - WRITE_ONLINE
  - WRITE_OFFLINE
  policy:
    roleBasedPolicy:
      roles:
      - store_admin
meta:
  createdTimestamp: '2024-09-09T06:41:28.335684Z'
  lastUpdatedTimestamp: '2024-09-09T06:41:28.335684Z'



List the roles specified by these permissions.

In [31]:
!feast -c rbac/feature_repo permissions list-roles

+--------------+
| ROLE NAME    |
+==============+
| batch_admin  |
+--------------+
| fresh_writer |
+--------------+
| reader       |
+--------------+
| store_admin  |
+--------------+


For each configured role, list all the resources and operations that are allowed to a user impersonating this role.

In [32]:
!feast -c rbac/feature_repo permissions list-roles -v

ROLE NAME     RESOURCE NAME                RESOURCE TYPE        PERMITTED ACTIONS
batch_admin   driver                       Entity               -
batch_admin   driver_hourly_stats          FeatureView          CREATE
                                                                DESCRIBE
                                                                UPDATE
                                                                DELETE
                                                                READ_OFFLINE
                                                                WRITE_OFFLINE
batch_admin   driver_hourly_stats_fresh    FeatureView          CREATE
                                                                DESCRIBE
                                                                UPDATE
                                                                DELETE
                                                                READ_OFFLINE
                                                